<a href="https://colab.research.google.com/github/vimalk2516/vimalgbt/blob/main/ai_physics_teacher_llm_dolly3b_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dolly 3B Model - by Databricks
---
1. Model Name: Dolly-v2-3b
2. Model Parameters: 3 Billion
3. Training: Instruction-tuned Model
4. Link: https://huggingface.co/databricks/dolly-v2-3b
---

In [ ]:
# install dependencies
!pip install transformers
!pip install sentencepiece
!pip install accelerate

In [ ]:
import torch                        # allows Tensor computation with strong GPU acceleration
from transformers import pipeline   # fast way to use pre-trained models for inference
import os

In [ ]:
# load model
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                            torch_dtype=torch.bfloat16,
                            trust_remote_code=True,
                            device_map="auto")

In [ ]:
# define helper function
def get_completion_dolly(input):
  system = f"""
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  """
  prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from Dolly-v2-3b:"
  print(prompt)
  dolly_response = dolly_pipeline(prompt,
                                  max_new_tokens=500
                                  )
  return dolly_response[0]["generated_text"]

In [ ]:
# let's prompt
prompt = "Explain the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"

print(get_completion_dolly(prompt))

#### System: 
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  
#### User: 
Explain the difference between nuclear fission and fusion.

#### Response from Dolly-v2-3b:


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 27906 has 14.74 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 17.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
from transformers import pipeline
import os

# Load model, offloading to CPU and clearing cache
try:
    torch.cuda.empty_cache()  # Clear GPU cache
    dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                              torch_dtype=torch.bfloat16,
                              trust_remote_code=True,
                              device_map="cpu")  # Try running on CPU
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("CUDA out of memory. Switching to CPU and reducing max_new_tokens.")
        # Consider using smaller model and reduce sequence length
        dolly_pipeline = pipeline(model="databricks/dolly-v2-12b", # Try a smaller Dolly version if available
                              torch_dtype=torch.bfloat16,
                              trust_remote_code=True,
                              device_map="cpu")
    else:
        raise e

# define helper function
def get_completion_dolly(input):
    system = f"""
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    """
    prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from Dolly-v2-3b:"
    print(prompt)

    # Reduced max_new_tokens
    dolly_response = dolly_pipeline(prompt,
                                    max_new_tokens=250) # Reduce output sequence length if necessary

    return dolly_response[0]["generated_text"]

# let's prompt
prompt = "Explain the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"

print(get_completion_dolly(prompt))

In [ ]:
import torch
from transformers import pipeline
import os

# Load model, offloading to CPU and clearing cache
try:
    torch.cuda.empty_cache()  # Clear GPU cache
    dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                              torch_dtype=torch.bfloat16,
                              trust_remote_code=True,
                              device_map="cpu")  # Try running on CPU
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("CUDA out of memory. Switching to CPU and reducing max_new_tokens.")
        # Consider using smaller model and reduce sequence length
        dolly_pipeline = pipeline(model="databricks/dolly-v2-12b", # Try a smaller Dolly version if available
                              torch_dtype=torch.bfloat16,
                              trust_remote_code=True,
                              device_map="cpu")
    else:
        raise e

# define helper function
def get_completion_dolly(input):
    system = f"""
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    """
    prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from Dolly-v2-3b:"
    print(prompt)

    # Reduced max_new_tokens
    dolly_response = dolly_pipeline(prompt,
                                    max_new_tokens=250) # Reduce output sequence length if necessary

    return dolly_response[0]["generated_text"]

# let's prompt
prompt = "Explain the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"

print(get_completion_dolly(prompt))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### System: 
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    
#### User: 
Explain the difference between nuclear fission and fusion.

#### Response from Dolly-v2-3b:
Nuclear fission occurs when a subatomic particle smashes into a large nucleus, separating the atom in two pieces. It creates two smaller pieces which are also atoms. 

Nuclear fusion occurs when two or more smaller atoms come together to create one larger atom. It requires two or more atoms with different isotopes, or types of an element, and the energy created from the heat of fusion can power a small device.


### Build AI Physics Tutor

In [ ]:
!pip install gradio

In [ ]:
import locale
# print(locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"

# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

In [ ]:
import gradio as gr

# build an app front-end / ai tool with Gradio
iface = gr.Interface(fn=get_completion_dolly, inputs=[gr.Textbox(label="Insert Prompt Here",lines=6)],
                     outputs=[gr.Textbox(label="Your Answer Here",lines=3)],
                     title="My AI Physics Teacher",
                     examples=["Explain the difference between nuclear fusion and fission.",
                              "Why is the Sky blue?"]
                     )
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a4211a2595d7212d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
iface.close()

## Falcon 7B Model - by TII
---
1. Model Name: Falcon-7b-instruct
2. Model Parameters: 7 Billion
3. Training: Instruction-tuned Model
4. Link: https://huggingface.co/tiiuae/falcon-7b-instruct
---

In [ ]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
# define completion function
def get_completion_falcon(input):
  # prompt = f"You are an expert Physicist. Your job is to explain complex Physics concepts in simple words. \n{input}"
  # print(prompt)
  system = f"""
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  """
  prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
  print(prompt)
  falcon_response = falcon_pipeline(prompt,
                                    max_length=500,
                                    do_sample=True,
                                    top_k=10,
                                    num_return_sequences=1,
                                    eos_token_id=tokenizer.eos_token_id,
                                    )

  return falcon_response

In [ ]:
# let's prompt
prompt = "Explain to me the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"
response = get_completion_falcon(prompt)
print(response[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


#### System: 
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  
#### User: 
Explain to me the difference between nuclear fission and fusion.

#### Response from falcon-7b-instruct:
#### System: 
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  
#### User: 
Explain to me the difference between nuclear fission and fusion.

#### Response from falcon-7b-instruct:
Nuclear fission is the splitting of an atom into two smaller nuclei, releasing energy in the form of gamma rays, whereas nuclear fusion is the combination of two small nuclei to form a large one releasing energy. In nuclear fission, a neutron is introduced into the nucleus of an atom, which causes the nucleus to break and release a large amount of energy. In nuclear fusion, two hydrogen nuclei combine, releasing energy due to the binding energy difference.


In [ ]:
print(response[0]['generated_text'])

#### System: 
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  
#### User: 
Explain to me the difference between nuclear fission and fusion.

#### Response from falcon-7b-instruct:
Nuclear fission is the splitting of an atom into two smaller nuclei, releasing energy in the form of gamma rays, whereas nuclear fusion is the combination of two small nuclei to form a large one releasing energy. In nuclear fission, a neutron is introduced into the nucleus of an atom, which causes the nucleus to break and release a large amount of energy. In nuclear fusion, two hydrogen nuclei combine, releasing energy due to the binding energy difference.


## Stable Diffusion v1-5 - by RunwayML
---
1. Model Name: stable-diffusion-v1-5
2. Model type: Diffusion-based text-to-image generation model
3. Model Description: Generate/ modify images w/ text prompts.
4. Link: https://huggingface.co/runwayml/stable-diffusion-v1-5
---

In [ ]:
# Developed by: Robin Rombach, Patrick Esser
# Latent Diffusion Model that uses a fixed, pretrained text encoder (CLIP ViT-L/14) as suggested in the Imagen paper.
# Language(s): English
# License: The CreativeML OpenRAIL M license is an Open RAIL M license, adapted from the work that
# BigScience and the RAIL Initiative are jointly carrying in the area of responsible AI licensing.
# See also the article about the BLOOM Open RAIL license on which our license is based.
# https://bigscience.huggingface.co/blog/the-bigscience-rail-license

In [ ]:
# install dependencies
!pip install diffusers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
def get_completion_sd(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

In [ ]:
#let's prompt

# prompt = "astronaut, riding a horse, on mars, human colony"
# prompt = "children, playing in disneyland, view from a distance"
prompt = """baby llama, wearing red muffler,
grazing, open field, sunset
"""
print(prompt)
sd_image = get_completion_sd(prompt)
sd_image.save("./llama.jpg")

baby llama, wearing red muffler,
grazing, open field, sunset



  0%|          | 0/50 [00:00<?, ?it/s]

### Build AI Image Generator

In [ ]:
!pip install gradio==3.48.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 12.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl 

In [ ]:
import gradio as gr

In [ ]:
def get_completion(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

# def generate(prompt):
#     output = get_completion_sd(prompt)
#     return output

genai_app = gr.Interface(fn=get_completion,
                         inputs=[gr.Textbox(label="Your prompt")],
                         outputs=[gr.Image(label="Result")],
                         title="Generate Cool Images",
                         description="Generate any image with Stable Diffusion",
                         allow_flagging="never",
                         examples=["astronaut, riding a horse, on mars",
                                   "cargo ship, flying, in space"])
genai_app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d1b6647af604f61598.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# gr.close_all()